In [1]:
# Importando as Bibiotecas
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display, Javascript
from IPython.display import Image
from google.colab.output import eval_js
from base64 import b64decode

In [2]:
# Função de Exibição
def imshow(title = "Image", image = None, size = 8):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

## **Captura da Imagem via Webcam**

In [ ]:
# Função criada para tirar fotos via WebCam
def take_photo(filename='photo.jpg', quality=0.8):

  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Redimensionando a saída para ajustar o elemento de vídeo.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Aguarde o clique em Capture.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])

  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
try:
  filename = take_photo()
  print('Salvo em {}'.format(filename))
  
  # Mostrando a imagem que acabou de ser tirada.
  display(Image(filename))
except Exception as err:
  # Erros lançados se o usuário não tiver uma webcam ou se não tiver concedido permissão à página para acessá-la.
  print(str(err))

## **Detecção Olhos e Faces - Classificador HaarCascade**

In [ ]:
# Carrengando os Modelos de Classificadores
face_classifier = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('/content/haarcascade_eye.xml')

In [ ]:
# Carregando imagem e Convertendo em Escala de Cinza
img = cv2.imread('photo.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Deteção de Faces
faces = face_classifier.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors = 5)

# Caso para nenhuma face detectada
if faces is ():
    print("Nenhuma Face Encontrada")

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(127,0,255),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_classifier.detectMultiScale(roi_gray, 1.2, 3)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)

imshow('Olhos e Faces Detecadas', img)